In [89]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                          Dropout, Rescaling, RandomFlip, RandomRotation, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import zipfile
import os
import glob
import shutil

In [90]:
# fig = plt.figure(figsize=(16, 8))
# for i, path in enumerate(train_list[:10], 1):
#     subplot = fig.add_subplot(2, 5, i)
#     subplot.set_title('%s' %path.split('/')[1].split('.')[0])
#     img = cv2.imread(path)[...,::-1]
#     img = cv2.resize(img, (224,224))
#     plt.imshow(img)

In [91]:
img_height,img_width = 224, 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'E:/4--2/ML_PROJECT/Broken-Working-Website-Classification-Using-CNN/code/dataset/',
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True,
  label_mode='binary')


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'E:/4--2/ML_PROJECT/Broken-Working-Website-Classification-Using-CNN/code/dataset/',
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True,
  label_mode='binary')

Found 15726 files belonging to 2 classes.
Using 12581 files for training.
Found 15726 files belonging to 2 classes.
Using 3145 files for validation.


In [92]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [93]:
train_ds.class_names

['broken', 'non-broken']

In [94]:
rescale = Sequential([
  Rescaling(1./255)
])

In [95]:
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])

In [96]:
# image = tf.expand_dims(img, 0)
# plt.figure(figsize=(10, 10))
# for i in range(9):
#   augmented_image = data_augmentation(image)
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(augmented_image[0])
#   plt.axis("off")

In [97]:
base_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet', classes=2)
for layer in base_model.layers:
        layer.trainable=False

In [98]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [101]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(1e-4)))

In [102]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_10 (Flatten)        (None, 2048)              0         
                                                                 
 dense_17 (Dense)            (None, 256)               524544    
                                                                 
 batch_normalization_8 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_18 (Dense)            (None, 1)                 257       
                                                                 
Total params: 24,113,537
Trainable params: 525,313
Non-trainable params: 23,588,224
___________________________________

In [103]:
early_stop = EarlyStopping(monitor="val_loss",min_delta=0, patience=5,
                           verbose=0, mode="min", baseline=None, restore_best_weights=True)
check = ModelCheckpoint(filepath='convnet.keras', save_best_only=True, monitor='val_loss')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [104]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=SGD(learning_rate = 0.001, momentum = 0.1),
              metrics=['accuracy'])

In [105]:
num_epoch = 10
model_log = model.fit(train_ds,
                      epochs=num_epoch,
                      verbose=1,
                      validation_data=val_ds,
                      callbacks = [early_stop, check, learning_rate_reduction])
model.save_weights('weights.h5')

Epoch 1/10
394/394 [==============================] - 862s 2s/step - loss: 0.0027 - accuracy: 0.8360 - val_loss: 0.0040 - val_accuracy: 0.8534 - lr: 0.0010
Epoch 2/10
394/394 [==============================] - 1069s 3s/step - loss: 0.0056 - accuracy: 0.8586 - val_loss: 0.0075 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 3/10
394/394 [==============================] - 1031s 3s/step - loss: 0.0095 - accuracy: 0.8698 - val_loss: 0.0118 - val_accuracy: 0.8712 - lr: 0.0010
Epoch 4/10
394/394 [==============================] - ETA: 0s - loss: 0.0145 - accuracy: 0.8786
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
394/394 [==============================] - 920s 2s/step - loss: 0.0145 - accuracy: 0.8786 - val_loss: 0.0175 - val_accuracy: 0.8779 - lr: 0.0010
Epoch 5/10
394/394 [==============================] - 906s 2s/step - loss: 0.0191 - accuracy: 0.8863 - val_loss: 0.0209 - val_accuracy: 0.8913 - lr: 5.0000e-04
Epoch 6/10
394/394 [==============================] -